In [1]:
#import libraries
import pandas as pd
print('pandas: %s' % pd.__version__)

import xarray as xr
print('xarray: %s' % xr.__version__)

pd.options.display.max_columns = None
pd.set_option('display.max_rows', 150)

import numpy as np
print('geopandas: %s' % np.__version__)



# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version



# Other utilities
import sys
import os

# Assign main directory to a variable
main_dir=os.path.dirname(sys.path[0])
print(main_dir)

pandas: 1.5.2
xarray: 2022.12.0
geopandas: 1.23.5
pandas: 1.5.2
numpy: 1.23.5
/Users/3rfanian/P_main


In [2]:
def duration_year(y):
    y_s = (y*10000) + 101
    y_e = (y*10000) + 1231
    y_s2e =  str(y_s) +"_"+ str(y_e)
    return y_s2e

In [3]:
def data_fram_years(year_start,year_end):
    paths_clim = "9var/tc_clim_obs_48_17_daily_smooth_0.25_1980_2022_2.5.grb"
    clim  = xr.open_dataset(paths_clim, engine='cfgrib')
    clim = clim.to_dataframe()
    clim = clim.reset_index()
    fram_lists = list()
    for i in range(year_start,year_end+1):
        clim2 = clim.loc[(clim.time>=f'{i}-01-01') & (clim.time<f'{i+1}-01-01')]
        clim2 = clim2.loc[:,['time','latitude','longitude','p131089']]
        clim2 = clim2.rename(columns={'p131089': 'clim'})
        clim2['longitude'] = (clim2['longitude'] + 180) % 360 - 180
        df = data_frame_year(i).merge(clim2)
        fram_lists.append(df)
        print(i)
    result = pd.concat(fram_lists)
    return result


In [44]:
def tom_generate(df):
    new_traning_sets = df
    new_traning_sets['Real_tom_target'] = new_traning_sets['new_target']
    rows, cols =  len(df.latitude.unique()), len(df.longitude.unique())
    size_rc = rows * cols
    for i in range(0,int((new_traning_sets.shape[0]/size_rc)-1)):
        j=i+1
        each_row2 = new_traning_sets.iloc[size_rc*j:size_rc*(j+1),:]
        new_traning_sets.iloc[size_rc*i:size_rc*(i+1),13:14] = each_row2['new_target'].values
    print("======== Done ========")
    return new_traning_sets

In [5]:
 def data_frame_year(year_file):
    supermerge=[]
    if year_file in range(1980,2020):
        
        part1 = "tc_data/{}".format(duration_year(year_file))
        part2 = part1+".grb"
        paths = "{}".format(part2)

        ds_vo = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'vo'}})
        gpi_bidim_vo = ds_vo.to_dataframe()
        gpi_bidim_vo = gpi_bidim_vo.reset_index()
        gpi_bidim_vo = gpi_bidim_vo.loc[:,['time','latitude','longitude','vo']]

        ds_r = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'r'}})
        gpi_bidim_r = ds_r.to_dataframe()
        gpi_bidim_r = gpi_bidim_r.reset_index()
        gpi_bidim_r = gpi_bidim_r.loc[:,['time','latitude','longitude','r']]

        ds_u200 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'u'}})
        gpi_bidim_u200 = ds_u200.to_dataframe()
        gpi_bidim_u200 = gpi_bidim_u200.reset_index()
        gpi_bidim_u200 = gpi_bidim_u200.loc[gpi_bidim_u200.isobaricInhPa==200.0,['time','latitude','longitude','u']]
        gpi_bidim_u200.rename(columns = {'u':'u_200'}, inplace = True)

        ds_u850 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'u'}})
        gpi_bidim_u850 = ds_u850.to_dataframe()
        gpi_bidim_u850 = gpi_bidim_u850.reset_index()
        gpi_bidim_u850 = gpi_bidim_u850.loc[gpi_bidim_u850.isobaricInhPa==850.0,['time','latitude','longitude','u']]
        gpi_bidim_u850.rename(columns = {'u':'u_850'}, inplace = True)

        ds_v200 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'v'}})
        gpi_bidim_v200 = ds_v200.to_dataframe()
        gpi_bidim_v200 = gpi_bidim_v200.reset_index()
        gpi_bidim_v200 = gpi_bidim_v200.loc[gpi_bidim_v200.isobaricInhPa==200.0,['time','latitude','longitude','v']]
        gpi_bidim_v200.rename(columns = {'v':'v_200'}, inplace = True)

        ds_v850 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'v'}})
        gpi_bidim_v850 = ds_v850.to_dataframe()
        gpi_bidim_v850 = gpi_bidim_v850.reset_index()
        gpi_bidim_v850 = gpi_bidim_v850.loc[gpi_bidim_v850.isobaricInhPa==850.0,['time','latitude','longitude','v']]
        gpi_bidim_v850.rename(columns = {'v':'v_850'}, inplace = True)

        part1_ttr = "tc_data/an_{}".format(duration_year(year_file))
        part2_ttr = part1_ttr+"_25"+".grb"
        paths_ttr = "{}".format(part2_ttr)
        ds_ttr = xr.open_dataset(paths_ttr, engine='cfgrib')
        gpi_bidim_ttr = ds_ttr.to_dataframe()
        gpi_bidim_ttr = gpi_bidim_ttr.reset_index()
        gpi_bidim_ttr = gpi_bidim_ttr.loc[:,['time','latitude','longitude','ttr']]
        gpi_bidim_ttr['longitude'] = (gpi_bidim_ttr['longitude'] + 180) % 360 - 180

        part1_sst = "tc_data/2d_sst_trop_{}".format(duration_year(year_file))
        part2_sst = part1_sst+".grb"
        paths_sst = "{}".format(part2_sst)
        ds_sst = xr.open_dataset(paths_sst, engine='cfgrib')
        gpi_bidim_sst = ds_sst.to_dataframe()
        gpi_bidim_sst = gpi_bidim_sst.reset_index()
        gpi_bidim_sst['sst'] = gpi_bidim_sst['sst'].fillna(0)
        gpi_bidim_sst = gpi_bidim_sst.loc[:,['time','latitude','longitude','sst']]

        '''
        part1_target = "tc_data/tcact_sin_year/actmask_hist_300km_17ms_sin_"
        i2s = str(year_file)
        path_target = part1_target+i2s+".grb"
        ds_lsm = xr.open_dataset(path_target, engine='cfgrib')
        gpi_bidim_lsm = ds_lsm.to_dataframe()
        gpi_bidim_lsm = gpi_bidim_lsm.reset_index()
        gpi_bidim_lsm = gpi_bidim_lsm.loc[:,['time','latitude','longitude','lsm']]
        
        part1_newtarget = "actmask_hist_300km_17ms_2_2.5/actmask_hist_300km_17ms_2_"
        i2s = str(year_file)
        path_newtarget = part1_newtarget+i2s+"_2.5.grb"
        ds_newlsm = xr.open_dataset(path_newtarget, engine='cfgrib')
        gpi_bidim_newlsm = ds_newlsm.to_dataframe()
        gpi_bidim_newlsm = gpi_bidim_newlsm.reset_index()
        gpi_bidim_newlsm = gpi_bidim_newlsm.loc[:,['time','latitude','longitude','lsm']]
        gpi_bidim_newlsm = gpi_bidim_newlsm.rename(columns={'lsm': 'newlsm'})
        '''
        part1_ism1 = "tc_data/tc_target/2.5col_obs_{}".format(year_file)
        part2_ism1 = part1_ism1+"_48_17.grb"
        paths_ism1 = "{}".format(part2_ism1)
        ds_ism1  = xr.open_dataset(paths_ism1, engine='cfgrib')
        gpi_bidim_ism1 = ds_ism1.to_dataframe()
        gpi_bidim_ism1 = gpi_bidim_ism1.reset_index()
        gpi_bidim_ism1 = gpi_bidim_ism1.loc[:,['time','latitude','longitude','p131089']]
        gpi_bidim_ism1 = gpi_bidim_ism1.rename(columns={'p131089': 'new_target'})
        #gpi_bidim_ism11 = gpi_bidim_ism1[gpi_bidim_ism1['latitude'].isin(lats) & gpi_bidim_ism1['longitude'].isin(longs)]
        
        supermerge = gpi_bidim_vo.merge(gpi_bidim_r).merge(gpi_bidim_u200).merge(gpi_bidim_u850).merge(gpi_bidim_v200).merge(gpi_bidim_v850).merge(gpi_bidim_ttr).merge(gpi_bidim_sst).merge(gpi_bidim_ism1)

    if year_file in range(2020,2023):
        if year_file==2020:
            paths = 'tc_data/serv/2d_gpi_20200101_20201231.grb'
        elif year_file==2021:
            paths = 'tc_data/serv/2d_gpi_20210101_20211231.grb'
        elif year_file==2022:
            paths = 'tc_data/serv/2d_gpi_20220101_20221231.grb'
        else:
            print(' ******************** ')
        print(paths)
        ds_vo = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'vo'}})
        gpi_bidim_vo = ds_vo.to_dataframe()
        gpi_bidim_vo = gpi_bidim_vo.reset_index()
        gpi_bidim_vo = gpi_bidim_vo.loc[:,['time','latitude','longitude','vo']]

        ds_r = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'r'}})
        gpi_bidim_r = ds_r.to_dataframe()
        gpi_bidim_r = gpi_bidim_r.reset_index()
        gpi_bidim_r = gpi_bidim_r.loc[:,['time','latitude','longitude','r']]

        ds_u200 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'u'}})
        gpi_bidim_u200 = ds_u200.to_dataframe()
        gpi_bidim_u200 = gpi_bidim_u200.reset_index()
        gpi_bidim_u200 = gpi_bidim_u200.loc[gpi_bidim_u200.isobaricInhPa==200.0,['time','latitude','longitude','u']]
        gpi_bidim_u200.rename(columns = {'u':'u_200'}, inplace = True)

        ds_u850 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'u'}})
        gpi_bidim_u850 = ds_u850.to_dataframe()
        gpi_bidim_u850 = gpi_bidim_u850.reset_index()
        gpi_bidim_u850 = gpi_bidim_u850.loc[gpi_bidim_u850.isobaricInhPa==850.0,['time','latitude','longitude','u']]
        gpi_bidim_u850.rename(columns = {'u':'u_850'}, inplace = True)

        ds_v200 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'v'}})
        gpi_bidim_v200 = ds_v200.to_dataframe()
        gpi_bidim_v200 = gpi_bidim_v200.reset_index()
        gpi_bidim_v200 = gpi_bidim_v200.loc[gpi_bidim_v200.isobaricInhPa==200.0,['time','latitude','longitude','v']]
        gpi_bidim_v200.rename(columns = {'v':'v_200'}, inplace = True)

        ds_v850 = xr.open_dataset(paths, engine='cfgrib',backend_kwargs={'filter_by_keys': {'shortName': 'v'}})
        gpi_bidim_v850 = ds_v850.to_dataframe()
        gpi_bidim_v850 = gpi_bidim_v850.reset_index()
        gpi_bidim_v850 = gpi_bidim_v850.loc[gpi_bidim_v850.isobaricInhPa==850.0,['time','latitude','longitude','v']]
        gpi_bidim_v850.rename(columns = {'v':'v_850'}, inplace = True)


        part1_ttr = "tc_data/serv/an_{}".format(duration_year(year_file))
        part2_ttr = part1_ttr+"_25"+".grb"
        paths_ttr = "{}".format(part2_ttr)
        ds_ttr = xr.open_dataset(paths_ttr, engine='cfgrib')
        gpi_bidim_ttr = ds_ttr.to_dataframe()
        gpi_bidim_ttr = gpi_bidim_ttr.reset_index()
        gpi_bidim_ttr = gpi_bidim_ttr.loc[:,['time','latitude','longitude','ttr']]
        gpi_bidim_ttr['longitude'] = (gpi_bidim_ttr['longitude'] + 180) % 360 - 180

        part1_sst = "tc_data/serv/2d_sst_trop_{}".format(duration_year(year_file))
        part2_sst = part1_sst+".grb"
        paths_sst = "{}".format(part2_sst)
        ds_sst = xr.open_dataset(paths_sst, engine='cfgrib')
        gpi_bidim_sst = ds_sst.to_dataframe()
        gpi_bidim_sst = gpi_bidim_sst.reset_index()
        gpi_bidim_sst['sst'] = gpi_bidim_sst['sst'].fillna(0)
        gpi_bidim_sst = gpi_bidim_sst.loc[:,['time','latitude','longitude','sst']]
        

        part1_ism1 = "tc_data/tc_target/2.5col_obs_{}".format(year_file)
        part2_ism1 = part1_ism1+"_48_17.grb"
        paths_ism1 = "{}".format(part2_ism1)
        ds_ism1  = xr.open_dataset(paths_ism1, engine='cfgrib')
        gpi_bidim_ism1 = ds_ism1.to_dataframe()
        gpi_bidim_ism1 = gpi_bidim_ism1.reset_index()
        gpi_bidim_ism1 = gpi_bidim_ism1.loc[:,['time','latitude','longitude','p131089']]
        gpi_bidim_ism1 = gpi_bidim_ism1.rename(columns={'p131089': 'new_target'})
        #gpi_bidim_ism11 = gpi_bidim_ism1[gpi_bidim_ism1['latitude'].isin(lats) & gpi_bidim_ism1['longitude'].isin(longs)]
        supermerge = gpi_bidim_vo.merge(gpi_bidim_r).merge(gpi_bidim_u200).merge(gpi_bidim_u850).merge(gpi_bidim_v200).merge(gpi_bidim_v850).merge(gpi_bidim_ttr).merge(gpi_bidim_sst).merge(gpi_bidim_ism1)
        #supermerge = supermerge[supermerge['latitude'].isin(lats) & supermerge['longitude'].isin(longs)]
        
    return supermerge

In [17]:
train = data_fram_years(1980,1990)


1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990


In [18]:
train

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,new_target,clim
0,1980-01-01,90.0,-180.0,-0.000007,47.091389,10.406403,10.991116,4.897507,-0.683846,-171.173340,271.459717,0.0,0.0
1,1980-01-01,90.0,-177.5,-0.000007,47.124294,10.544151,10.880585,4.430099,-1.140003,-171.134811,271.459717,0.0,0.0
2,1980-01-01,90.0,-175.0,-0.000006,47.184170,10.662994,10.752586,3.961662,-1.587284,-171.019257,271.459717,0.0,0.0
3,1980-01-01,90.0,-172.5,-0.000007,47.268673,10.763901,10.608711,3.492065,-2.024754,-170.942215,271.459717,0.0,0.0
4,1980-01-01,90.0,-170.0,-0.000007,47.374889,10.847244,10.449337,3.022316,-2.453190,-170.927414,271.459717,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836875,1990-12-31,-90.0,167.5,0.000013,102.533356,0.131187,2.056641,6.348770,2.858948,-186.617783,0.000000,0.0,0.0
3836876,1990-12-31,-90.0,170.0,0.000013,102.477448,-0.132439,1.977795,6.298615,2.942482,-186.635559,0.000000,0.0,0.0
3836877,1990-12-31,-90.0,172.5,0.000013,102.412720,-0.390495,1.895233,6.236923,3.019199,-186.659256,0.000000,0.0,0.0
3836878,1990-12-31,-90.0,175.0,0.000012,102.340973,-0.642502,1.810452,6.164047,3.089214,-186.813339,0.000000,0.0,0.0


In [20]:
#train = data_fram_years(1980,2010)
valid = data_fram_years(2011,2015)
#test = data_fram_years(2016,2022)

2011
2012
2013
2014
2015


In [18]:
 tt = test
tt

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,new_target,clim
0,2016-01-01,90.0,-180.0,-0.000055,63.153320,-9.171402,-1.816597,-13.858635,5.202774,-166.953339,271.459473,0.0,0.0
1,2016-01-01,90.0,-177.5,-0.000055,62.559265,-9.667664,-1.664066,-13.267357,5.269211,-166.923706,271.459473,0.0,0.0
2,2016-01-01,90.0,-175.0,-0.000055,61.945679,-10.136597,-1.500095,-12.661011,5.333832,-166.734070,271.459473,0.0,0.0
3,2016-01-01,90.0,-172.5,-0.000055,61.326180,-10.576843,-1.324284,-12.041168,5.394989,-166.615555,271.459473,0.0,0.0
4,2016-01-01,90.0,-170.0,-0.000056,60.712692,-10.988846,-1.136375,-11.409485,5.451176,-166.574081,271.459473,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836875,2022-12-31,-90.0,167.5,0.000016,97.887581,-1.422318,2.364288,-3.507492,0.720871,-182.426666,0.000000,0.0,0.0
3836876,2022-12-31,-90.0,170.0,0.000016,97.870003,-1.215286,2.370148,-3.650070,0.809738,-182.444443,0.000000,0.0,0.0
3836877,2022-12-31,-90.0,172.5,0.000017,97.856331,-1.002396,2.372101,-3.778976,0.898605,-182.462219,0.000000,0.0,0.0
3836878,2022-12-31,-90.0,175.0,0.000017,97.844612,-0.783646,2.370148,-3.892258,0.987473,-182.586670,0.000000,0.0,0.0


In [19]:
def move_col(df, name_index, new_index):
    col = df.pop(f"{name_index}")
    df.insert(new_index, f"{name_index}", col)
    return df

In [45]:
df1 = move_col(train,'clim',11)
#df2 = move_col(valid,'clim',11)
#df3 = move_col(test,'clim',11)
df1

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,1980-01-01,90.0,-180.0,-0.000007,47.091389,10.406403,10.991116,4.897507,-0.683846,-171.173340,271.459717,0.0,0.0,0.0
1,1980-01-01,90.0,-177.5,-0.000007,47.124294,10.544151,10.880585,4.430099,-1.140003,-171.134811,271.459717,0.0,0.0,0.0
2,1980-01-01,90.0,-175.0,-0.000006,47.184170,10.662994,10.752586,3.961662,-1.587284,-171.019257,271.459717,0.0,0.0,0.0
3,1980-01-01,90.0,-172.5,-0.000007,47.268673,10.763901,10.608711,3.492065,-2.024754,-170.942215,271.459717,0.0,0.0,0.0
4,1980-01-01,90.0,-170.0,-0.000007,47.374889,10.847244,10.449337,3.022316,-2.453190,-170.927414,271.459717,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836875,1990-12-31,-90.0,167.5,0.000013,102.533356,0.131187,2.056641,6.348770,2.858948,-186.617783,0.000000,0.0,0.0,0.0
3836876,1990-12-31,-90.0,170.0,0.000013,102.477448,-0.132439,1.977795,6.298615,2.942482,-186.635559,0.000000,0.0,0.0,0.0
3836877,1990-12-31,-90.0,172.5,0.000013,102.412720,-0.390495,1.895233,6.236923,3.019199,-186.659256,0.000000,0.0,0.0,0.0
3836878,1990-12-31,-90.0,175.0,0.000012,102.340973,-0.642502,1.810452,6.164047,3.089214,-186.813339,0.000000,0.0,0.0,0.0


In [46]:
df1 = tom_generate(df1)
#df2 = tom_generate(ds)
#df3 = tom_generate(test)

======== Done ========


In [40]:
df1

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
0,1980-01-01,90.0,-180.0,-0.000007,47.091389,10.406403,10.991116,4.897507,-0.683846,-171.173340,271.459717,0.0,0.0,0.0
1,1980-01-01,90.0,-177.5,-0.000007,47.124294,10.544151,10.880585,4.430099,-1.140003,-171.134811,271.459717,0.0,0.0,0.0
2,1980-01-01,90.0,-175.0,-0.000006,47.184170,10.662994,10.752586,3.961662,-1.587284,-171.019257,271.459717,0.0,0.0,0.0
3,1980-01-01,90.0,-172.5,-0.000007,47.268673,10.763901,10.608711,3.492065,-2.024754,-170.942215,271.459717,0.0,0.0,0.0
4,1980-01-01,90.0,-170.0,-0.000007,47.374889,10.847244,10.449337,3.022316,-2.453190,-170.927414,271.459717,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836875,1990-12-31,-90.0,167.5,0.000013,102.533356,0.131187,2.056641,6.348770,2.858948,-186.617783,0.000000,0.0,0.0,0.0
3836876,1990-12-31,-90.0,170.0,0.000013,102.477448,-0.132439,1.977795,6.298615,2.942482,-186.635559,0.000000,0.0,0.0,0.0
3836877,1990-12-31,-90.0,172.5,0.000013,102.412720,-0.390495,1.895233,6.236923,3.019199,-186.659256,0.000000,0.0,0.0,0.0
3836878,1990-12-31,-90.0,175.0,0.000012,102.340973,-0.642502,1.810452,6.164047,3.089214,-186.813339,0.000000,0.0,0.0,0.0


In [47]:

testt1 = df1.loc[(df1.new_target!=0.0) | (df1.Real_tom_target!=0.0)]
#testt1
tt = testt1.loc[(testt1.latitude==15.0)&(testt1.longitude==-180.0)&(testt1.time>='1980-01-01')&(testt1.time<'1980-05-01')]
tt

,time,latitude,longitude,vo,r,u_200,u_850,v_200,v_850,ttr,sst,clim,new_target,Real_tom_target
950400,1980-03-31,15.0,-180.0,-0.000008,58.941307,5.290474,-10.991375,1.399933,-2.330292,-291.045929,299.102844,0.0,0.0,42.5
960912,1980-04-01,15.0,-180.0,-0.000004,9.484217,6.042755,-15.012074,1.017593,-3.046165,-297.545563,299.121155,0.0,42.5,52.0
971424,1980-04-02,15.0,-180.0,-0.000002,11.314879,8.980865,-14.769081,6.928932,-2.125454,-298.108154,299.169220,0.0,52.0,51.0
981936,1980-04-03,15.0,-180.0,0.000004,12.410309,10.445465,-13.999081,15.456345,-2.379814,-271.341125,299.271851,0.0,51.0,3.5
992448,1980-04-04,15.0,-180.0,0.000009,7.393660,8.674187,-11.322742,15.042358,1.631676,-244.778519,299.378479,0.0,3.5,0.0


In [29]:
df1.to_csv('dataframes/Earth/9vars/training_earth_9var.csv')  
df2.to_csv('dataframes/Earth/9vars/valid_earth_9var.csv')  
df3.to_csv('dataframes/Earth/9vars/test_earth_9var.csv')  

In [ ]:
df1['shear'] = df1.apply(lambda x: np.sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
#df2['shear'] = df2.apply(lambda x: np.sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
#df3['shear'] = df3.apply(lambda x: np.sqrt((x.u_200-x.u_850)**2 + (x.v_200-x.v_850)**2),axis=1)
df1